In [1]:
from alpaca.models import model_library
import sympy as sp
from IPython import display # pretty printing in jupyter. Does not work on regular python

In [2]:
charge = sp.symbols(r'\mathcal{X}')
charge_u = sp.symbols(r'\mathcal{X}_u')
charge_d = sp.symbols(r'\mathcal{X}_d')

Models from [1610.07593](https://arxiv.org/abs/1610.07593) to check we obtain the same results

In [4]:
R1 = model_library.fermion('3','1',-1/3,charge)
R2 = model_library.fermion('3','1',2/3,charge)
R3 = model_library.fermion('3','2',1/6,charge)
R4 = model_library.fermion('3','2',-5/6,charge)
R5 = model_library.fermion('3','2',7/6,charge)
R6 = model_library.fermion('3','3',-1/3,charge)
R7 = model_library.fermion('3','3',2/3,charge)
R8 = model_library.fermion('3','3',-4/3,charge)
R9 = model_library.fermion('6_bar','1',-1/3,charge)
R10 = model_library.fermion('6_bar','1',2/3,charge)
R11 = model_library.fermion('6_bar','2',1/6,charge)
R12 = model_library.fermion('8','1',-1,charge)
R13 = model_library.fermion('8','2',-1/2,charge)
R14 = model_library.fermion('15','1',-1/3,charge)
R15 = model_library.fermion('15','1',2/3,charge)

list_of_fermions=[R1,R2,R3,R4,R5,R6,R7, R8, R9, R10, R11, R12, R13, R14, R15]

In [44]:
list_of_KSVZ_models: list[model_library.KSVZ_model] = []
for i, fermion in enumerate(list_of_fermions):
    name = 'R' + str(i+1)
    list_of_KSVZ_models.append(model_library.KSVZ_model(name, [fermion]))
text = ''
for models in list_of_KSVZ_models:
   text += models.model_name + '\n' + r'\\' #+ '\n~' + r'\\' + '\n'
   text += models.couplings_latex() + '\n' + r'\\' + '\n'
   text += r'E/N = ' + str(models.E_over_N()) + '\n' + r'\\' + '\n~' + r'\\' + '\n'
display.Math(text)

<IPython.core.display.Math object>

Models from [1705.05370](https://arxiv.org/abs/1705.05370) to check we obtain the same results

In [47]:
DFSZ1 = model_library.model('DFSZ-I', {'eR': charge, 'uR': charge, 'dR': charge})
DFSZ2 = model_library.model('DFSZ-II', {'eR': -charge, 'uR': charge, 'dR': charge})
DFSZ3a = model_library.model('DFSZ-IIIa', {'eR': -(2*charge_u+charge_d), 'uR': charge_u, 'dR': charge_d})
DFSZ3b = model_library.model('DFSZ-IIIb', {'eR': charge_d,  'uR': charge_u, 'dR': charge_d})
DFSZ3c = model_library.model('DFSZ-IIIc', {'eR': charge_u+2*charge_d, 'uR': charge_u, 'dR': charge_d})
DFSZ3d = model_library.model('DFSZ-IIId', {'eR': -charge_u, 'uR': charge_u, 'dR': charge_d})

list_of_DFSZ_models=[DFSZ1, DFSZ2, DFSZ3a, DFSZ3b, DFSZ3c, DFSZ3d]

In [48]:
text = ''
for models in list_of_DFSZ_models:
   text += models.model_name + '\n' + r'\\' #+ '\n~' + r'\\' + '\n'
   text += models.couplings_latex() + '\n' + r'\\' + '\n'
   text += r'E/N = ' + str(models.E_over_N()) + '\n' + r'\\' + '\n~' + r'\\' + '\n'
display.Math(text)

<IPython.core.display.Math object>